# Initial Data Analysis

In [2]:
import pandas as pd

In [3]:
# Load the data
task1_train = pd.read_json('Flares-dataset/5w1h_subtarea_1_train.json', lines=True)
task1_test = pd.read_json('Flares-dataset/5w1h_subtask_1_trial.json', lines=True)

In [4]:
# Check the shape of the DataFrame
print("Shape of the DataFrame train:", task1_train.shape)
print("Shape of the DataFrame test:", task1_test.shape)

Shape of the DataFrame train: (1585, 3)
Shape of the DataFrame test: (168, 3)


The training set has a total of 1585 samples and the test set 168.

In [5]:
task1_train.head(2)

,Id,Text,Tags
0,732,"Dos días, exactamente han pasado dos días desd...","[{'Tag_Start': 52, 'Tag_End': 59, '5W1H_Label'..."
1,219,"Pero no pasará nada, el PSOE no exigirá la dim...","[{'Tag_Start': 21, 'Tag_End': 28, '5W1H_Label'..."


In [6]:
# Ensure both dataframes have an 'Id' column
if 'Id' in task1_train.columns and 'Id' in task1_test.columns:
    # Identify duplicate IDs
    duplicates_mask = task1_test['Id'].isin(task1_train['Id'])
    duplicate_ids = task1_test[duplicates_mask]

    # Print the number of duplicates
    print(f"Number of duplicates found: {duplicate_ids.shape[0]}")

    # Print the actual duplicate entries
    if duplicate_ids.shape[0] > 0:
        print("Duplicate Entries:")
        print(duplicate_ids['Id'])
    else:
        print("No duplicate entries found.")
else:
    print("Error: 'id' column not found in one or both DataFrames.")

Number of duplicates found: 168
Duplicate Entries:
0       72
1       64
2       49
3      130
4       61
      ... 
163     23
164    129
165    102
166      0
167    104
Name: Id, Length: 168, dtype: int64


The test set is a subset of the training set. In order to make a honest evaluation, we will need to split the train set into a train and validation set.

In [7]:
# Ensure pandas does not truncate any text output
pd.set_option('display.max_colwidth', None)

In [8]:
# Filter the DataFrame to get the 'Text' for the entry with Id == 1476
text_data = task1_train[task1_train['Id'] == 956]['Text']

# Print the content
print(text_data)

589    El experto ha acompañado el mensaje con una fotografía de uno de los casos observados, en el que se ve una lengua manchada o descolorida. 
Name: Text, dtype: object


## Sentence extraction

We can Extract the text based on Tag_Start and Tag_End.

In [9]:
# Your text
text = task1_train['Text'][0]

In [10]:
# Loop through each tag in the first item of task1_train['Tags']
for tag in task1_train['Tags'][0]:
    # Extract the text based on Tag_Start and Tag_End
    extracted_text = text[tag['Tag_Start']:tag['Tag_End']]
    # Print the extracted text and the corresponding 5W1H label
    print(f"Extracted Text: {extracted_text}, 5W1H Label: {tag['5W1H_Label']}")

Extracted Text: Sánchez, 5W1H Label: WHO
Extracted Text: en rueda de prensa en la Moncloa, 5W1H Label: WHERE
Extracted Text: a España, 5W1H Label: WHO
Extracted Text: entre abril y septiembre, 5W1H Label: WHEN
Extracted Text: un total de 87 millones de vacunas, 5W1H Label: WHAT
Extracted Text: las mentiras, 5W1H Label: WHAT
Extracted Text: de Sánchez, 5W1H Label: WHO
Extracted Text: ese refrán que dice, 5W1H Label: WHAT


In [11]:
text

'Dos días, exactamente han pasado dos días desde que Sánchez compareciera en rueda de prensa en la Moncloa afirmando que a España llegarían, entre abril y septiembre, un total de 87 millones de vacunas para darnos cuenta de que las mentiras de Sánchez hacen bueno ese refrán que dice que “la mentira tiene las patas muy cortas”.'

We want something like this:

In [12]:
x = {"input": text, "output": "Text: Sánchez, 5W1H Label: WHO\nText: en rueda de prensa en la Moncloa, 5W1H Label: WHERE\nText: a España, 5W1H Label: WHO\nText: entre abril y septiembre, 5W1H Label: WHEN\nText: un total de 87 millones de vacunas, 5W1H Label: WHAT\nText: las mentiras, 5W1H Label: WHAT\nText: de Sánchez, 5W1H Label: WHO\nText: ese refrán que dice, 5W1H Label: WHAT"}

Example of possible output

In [13]:
print(x['output'])

Text: Sánchez, 5W1H Label: WHO
Text: en rueda de prensa en la Moncloa, 5W1H Label: WHERE
Text: a España, 5W1H Label: WHO
Text: entre abril y septiembre, 5W1H Label: WHEN
Text: un total de 87 millones de vacunas, 5W1H Label: WHAT
Text: las mentiras, 5W1H Label: WHAT
Text: de Sánchez, 5W1H Label: WHO
Text: ese refrán que dice, 5W1H Label: WHAT


Example of model output for Generative model:

In [14]:
formatted_data = []  # List to hold the formatted dictionaries

for i in range(len(task1_train['Text'])):
    # Constructing the 'input' part
    input_text = task1_train['Text'][i]
    
    # Initializing the 'output' part
    output_text = ''
    
    # Loop through each tag and append the extracted text and 5W1H label to 'output'
    for tag in task1_train['Tags'][i]:
        extracted_text = input_text[tag['Tag_Start']:tag['Tag_End']]
        output_text += f"Text: {extracted_text}, 5W1H Label: {tag['5W1H_Label']}\n"
    
    # Remove the last newline and space
    output_text = output_text.rstrip('\n ')
    
    # Append the formatted dictionary to the list
    formatted_data.append({'input': input_text, 'output': output_text})

# Now `formatted_data` contains all your formatted dictionaries
# Here's how you can print the first formatted dictionary to check
print(formatted_data[0]['output'])

Text: Sánchez, 5W1H Label: WHO
Text: en rueda de prensa en la Moncloa, 5W1H Label: WHERE
Text: a España, 5W1H Label: WHO
Text: entre abril y septiembre, 5W1H Label: WHEN
Text: un total de 87 millones de vacunas, 5W1H Label: WHAT
Text: las mentiras, 5W1H Label: WHAT
Text: de Sánchez, 5W1H Label: WHO
Text: ese refrán que dice, 5W1H Label: WHAT


## Create Train split dataset

Since the current test set is a subset of the training set, we will create a new, independent split from the training data to ensure a fair evaluation.

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
task1_train = pd.read_json('Flares-dataset/5w1h_subtarea_1_train.json', lines=True)

# Split the dataset into training and testing sets (80% training, 20% testing)
train_set, test_set = train_test_split(task1_train, test_size=0.2, random_state=42)

# Save the training set as JSON
train_set.to_json('Flares-dataset/5w1h_subtarea_1_train_train.json', orient='records', lines=True)

# Save the testing set as JSON
test_set.to_json('Flares-dataset/5w1h_subtarea_1_train_test.json', orient='records', lines=True)

# Create Prompt template for fine-tuning LLM

#### Create a Prompt with List-Type Output:

[en rueda de prensa en la Moncloa, WHERE]

[a España, WHO]

[entre abril y septiembre, WHEN]

[un total de 87 millones de vacunas, WHAT]

[las mentiras, WHAT]

[de Sánchez, WHO]

[ese refrán que dice, WHAT]

In [21]:
### Con ejemplo en formato de lista
def format_data_zero(df):
    formatted_data = []  # List to hold the formatted dictionaries

    instruction = ("Analiza el texto y extrae fragmentos significativos, asignando a cada uno la etiqueta más "
                   "adecuada según el tipo de información que representan en formato JSON. Las categorías de etiquetado son las siguientes:"
                   "\n\nWHO: Sujetos o entidades involucradas.\nWHAT: Hechos u objetos mencionados.\nWHEN: Detalles relacionados con el tiempo.\n"
                   "WHERE: Lugares mencionados.\nWHY: Causas o razones.\nHOW: Maneras o métodos descritos."
                   "WHY (para causas o razones), y HOW (para maneras o métodos descritos)."
                   "\n\n### Ejemplo:\nTexto: Dos días, exactamente han pasado dos días desde que Sánchez compareciera en rueda de prensa en la Moncloa afirmando que a España llegarían, "
                   "entre abril y septiembre, un total de 87 millones de vacunas para darnos cuenta de que las mentiras de Sánchez hacen bueno ese refrán que dice que "
                   "“la mentira tiene las patas muy cortas”.\nFormato [Texto, Etiqueta]:\n"
                   "[en rueda de prensa en la Moncloa, WHERE]\n[a España, WHO]\n[entre abril y septiembre, WHEN]\n[un total de 87 millones de vacunas, WHAT]"
                   "\n[las mentiras, WHAT]\n[de Sánchez, WHO]\n[ese refrán que dice, WHAT]\n\n### Texto:\n""")

    for i in range(len(df['Text'])):
        # Constructing the 'input' part with instruction
        input_text = instruction + df['Text'][i]
        
        # Initializing the 'output' part
        output_text = ''
        
        # Loop through each tag and append the extracted text and 5W1H label to 'output'
        for tag in df['Tags'][i]:
            extracted_text = input_text[tag['Tag_Start'] + len(instruction):tag['Tag_End'] + len(instruction)]
            output_text += f"[{extracted_text}, {tag['5W1H_Label']}]\n"
        
        # Remove the last newline and space
        output_text = output_text.rstrip('\n ')
        
        # Append the formatted dictionary to the list
        formatted_data.append({'input': input_text, 'output': output_text})

    return formatted_data

#### Create a Prompt with JSON Output (json_format1):

{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], 'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}

In [18]:
### Con ejemplo y en formato JSON
def format_data_zero(df):
    formatted_data = []  # List to hold the formatted dictionaries

    instruction = ("Tu tarea consiste en analizar el texto proporcionado y extraer fragmentos significativos, "
                   "transcribiéndolos exactamente como aparecen. Asigna a cada fragmento la etiqueta "
                   "correspondiente basada en la información que representa. Presenta los resultados en formato JSON. Las categorías de etiquetado son:"
                   "\n\nWHO: Sujetos o entidades involucradas.\nWHAT: Hechos u objetos mencionados.\nWHEN: Detalles relacionados con el tiempo.\n"
                   "WHERE: Lugares mencionados.\nWHY: Causas o razones.\nHOW: Maneras o métodos descritos."
                   "\n\nAbajo es un ejemplo:\n\nInput: Dos días, exactamente han pasado dos días desde que Sánchez compareciera en rueda de prensa en la Moncloa afirmando que a España llegarían, "
                   "entre abril y septiembre, un total de 87 millones de vacunas para darnos cuenta de que las mentiras de Sánchez hacen bueno ese refrán que dice que "
                   "“la mentira tiene las patas muy cortas”. Output: "
                   "{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], "
                   "'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}\n\nAhora, completa la siguiente tarea:\n\nInput: ")

    for i in range(len(df['Text'])):
        # Constructing the 'input' part with instruction
        input_text = instruction + df['Text'][i]
        
        # Initializing the 'output' part as a dictionary
        output_dict = {}
        
        # Loop through each tag and append the extracted text and 5W1H label to 'output'
        for tag in df['Tags'][i]:
            extracted_text = input_text[tag['Tag_Start'] + len(instruction):tag['Tag_End'] + len(instruction)]
            if tag['5W1H_Label'] not in output_dict:
                output_dict[tag['5W1H_Label']] = []
            output_dict[tag['5W1H_Label']].append(extracted_text)
        
        # Append the formatted dictionary to the list
        formatted_data.append({'input': input_text, 'output': output_dict, 'tags':df['Tags'][i], 'Id':int(df['Id'][i])})

    return formatted_data

#### Create a Prompt with JSON Output without example (json_format2):
We are goign to use the instruction, input and response format.

{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], 'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}

In [15]:
### Sin ejemplo y en formato JSON
def format_data_zero(df):
    formatted_data = []  # List to hold the formatted dictionaries
    
    instruction = """### Instruction:
Tu tarea es analizar el texto proporcionado para identificar y extraer fragmentos significativos, 
luego asignar a cada fragmento una etiqueta que describa la naturaleza de la información que contiene. 
Los resultados deben ser presentados en formato JSON.

Categorías de Etiquetado
Cada fragmento extraído debe clasificarse en una de las siguientes categorías:

WHAT (Qué): Señala los hechos, objetos, o cualquier elemento concreto mencionado en el texto.

WHO (Quién): Identifica a los sujetos o entidades (personas, organizaciones, etc.) que están 
involucrados o mencionados en el texto.

WHEN (Cuándo): Extrae detalles que especifican momentos o periodos de tiempo mencionados en el texto.

WHERE (Dónde): Localiza los lugares, espacios geográficos o direcciones mencionadas en el texto.

WHY (Por qué): Identifica las causas, razones o motivaciones explicadas en el texto.

HOW (Cómo): Describe las maneras, métodos o procedimientos que el texto detalla sobre cómo 
se realizan o suceden las cosas.

### Input:
"""
    
    for i in range(len(df['Text'])):
        # Constructing the 'input' part with instruction
        input_text = instruction + df['Text'][i]
        
        # Initializing the 'output' part as a dictionary
        output_dict = {}
        
        # Loop through each tag and append the extracted text and 5W1H label to 'output'
        for tag in df['Tags'][i]:
            extracted_text = input_text[tag['Tag_Start'] + len(instruction):tag['Tag_End'] + len(instruction)]
            if tag['5W1H_Label'] not in output_dict:
                output_dict[tag['5W1H_Label']] = []
            output_dict[tag['5W1H_Label']].append(extracted_text)
        
        # Append the formatted dictionary to the list
        formatted_data.append({'input': input_text, 'output': output_dict, 'tags':df['Tags'][i], 'Id':int(df['Id'][i])})

    return formatted_data

#### Create a Prompt with JSON Output without example (json_format2) SPANISH:
We are goign to use the instruction, input and response format.

{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], 'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}

In [49]:
### Sin ejemplo y en formato JSON
def format_data_zero(df):
    formatted_data = []  # List to hold the formatted dictionaries
    
    instruction = """A continuación, se presenta una instrucción que describe una tarea, junto con una entrada que proporciona contexto adicional. Escribe una respuesta que complete adecuadamente la solicitud.

### Instrucciones:
Tu tarea es analizar el texto proporcionado y extraer fragmentos que pertenezcan a las categorías específicas, asegurándote de que los fragmentos sean secuencias continuas de palabras exactamente como aparecen en el texto, sin reordenamientos ni omisiones.

1. Leer el Texto Completo:
Comprende el contenido general y el contexto del texto dado. Asegúrate de entender los principales eventos, descripciones, y entidades mencionadas.

2. Extraer Fragmentos Relevantes:
Identifica y extrae partes del texto que corresponden a cada una de las siguientes categorías. Los fragmentos deben ser exactos y no alterados del texto original.

3. Clasificar cada Fragmento:
Utiliza las categorías predefinidas para anotar cada fragmento extraído. Asegúrate de que la clasificación sea precisa según la definición de cada categoría.

Categorías:

WHAT (Qué): Hechos, objetos, o cualquier elemento concreto mencionado.
WHO (Quién): Sujetos o entidades (personas, organizaciones, etc.) mencionados.
WHEN (Cuándo): Momentos o periodos de tiempo especificados.
WHERE (Dónde): Lugares, espacios geográficos o direcciones.
WHY (Por qué): Causas, razones o motivaciones.
HOW (Cómo): Maneras, métodos o procedimientos detallados.

### Información de Entrada:
"""
    
    for i in range(len(df['Text'])):
        # Constructing the 'input' part with instruction
        input_text = instruction + df['Text'][i]
        
        # Initializing the 'output' part as a dictionary
        output_dict = {}
        
        # Loop through each tag and append the extracted text and 5W1H label to 'output'
        for tag in df['Tags'][i]:
            extracted_text = input_text[tag['Tag_Start'] + len(instruction):tag['Tag_End'] + len(instruction)]
            if tag['5W1H_Label'] not in output_dict:
                output_dict[tag['5W1H_Label']] = []
            output_dict[tag['5W1H_Label']].append(extracted_text)
        
        # Append the formatted dictionary to the list
        formatted_data.append({'input': input_text, 'output': output_dict, 'tags':df['Tags'][i], 'Id':int(df['Id'][i])})

    return formatted_data

Create a Prompt with JSON Output without example (json_format2) SPANISH:
We are goign to use the instruction, input and response format.

{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], 'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}

# Create training datasets with the prompt template format (train and trial)

In [19]:
# Load the data
task1_train = pd.read_json('Flares-dataset/5w1h_subtarea_1_train.json', lines=True)
task1_test = pd.read_json('Flares-dataset/5w1h_subtask_1_trial.json', lines=True)

In [20]:
# Create prompt following the prompt template
task1_zero_train = format_data_zero(task1_train)
task1_zero_test = format_data_zero(task1_test)

In [21]:
task1_zero_train[0].keys()

dict_keys(['input', 'output', 'tags', 'Id'])

In [22]:
import json

# Define the file path where you want to save the JSON data
file_path = 'data_files/5w1h_subtask_1_zero_train_json_format1.json'

# Save the `formatted_data` as a JSON file
with open(file_path, 'w') as f:
    json.dump(task1_zero_train, f, indent=4) 

In [23]:
import json

# Define the file path where you want to save the JSON data
file_path = 'data_files/5w1h_subtask_1_zero_test_json_format1.json'

# Save the `formatted_data` as a JSON file
with open(file_path, 'w') as f:
    json.dump(task1_zero_test, f, indent=4) 

## Dataset content

In [10]:
task1_zero_test = pd.read_json(file_path)

In [11]:
print(task1_zero_test['input'][0])

### Instrucciones:
Tu tarea es analizar el texto proporcionado para identificar y extraer fragmentos significativos, 
luego asignar a cada fragmento una etiqueta que describa la naturaleza de la información que contiene. 
Los resultados deben ser presentados en formato JSON.

Categorías de Etiquetado
Cada fragmento extraído debe clasificarse en una de las siguientes categorías:

WHAT (Qué): Señala los hechos, objetos, o cualquier elemento concreto mencionado en el texto.

WHO (Quién): Identifica a los sujetos o entidades (personas, organizaciones, etc.) que están 
involucrados o mencionados en el texto.

WHEN (Cuándo): Extrae detalles que especifican momentos o periodos de tiempo mencionados en el texto.

WHERE (Dónde): Localiza los lugares, espacios geográficos o direcciones mencionadas en el texto.

WHY (Por qué): Identifica las causas, razones o motivaciones explicadas en el texto.

HOW (Cómo): Describe las maneras, métodos o procedimientos que el texto detalla sobre cómo 
se realiz

In [12]:
task1_zero_test['output'][0]

{'WHAT': ['la incidencia acumulada'],
 'WHERE': ['en los 255,55 casos por cada 100.000 habitantes'],
 'WHEN': ['ayer, lunes 28 de diciembre de 2020']}

In [13]:
task1_zero_test['tags'][0]

[{'Tag_Start': 12,
  'Tag_End': 35,
  '5W1H_Label': 'WHAT',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'la incidencia acumulada'},
 {'Tag_Start': 64,
  'Tag_End': 111,
  '5W1H_Label': 'WHERE',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'en los 255,55 casos por cada 100.000 habitantes'},
 {'Tag_Start': 126,
  'Tag_End': 161,
  '5W1H_Label': 'WHEN',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'ayer, lunes 28 de diciembre de 2020'}]

In [14]:
print(task1_zero_test['input'][1])

### Instrucciones:
Tu tarea es analizar el texto proporcionado para identificar y extraer fragmentos significativos, 
luego asignar a cada fragmento una etiqueta que describa la naturaleza de la información que contiene. 
Los resultados deben ser presentados en formato JSON.

Categorías de Etiquetado
Cada fragmento extraído debe clasificarse en una de las siguientes categorías:

WHAT (Qué): Señala los hechos, objetos, o cualquier elemento concreto mencionado en el texto.

WHO (Quién): Identifica a los sujetos o entidades (personas, organizaciones, etc.) que están 
involucrados o mencionados en el texto.

WHEN (Cuándo): Extrae detalles que especifican momentos o periodos de tiempo mencionados en el texto.

WHERE (Dónde): Localiza los lugares, espacios geográficos o direcciones mencionadas en el texto.

WHY (Por qué): Identifica las causas, razones o motivaciones explicadas en el texto.

HOW (Cómo): Describe las maneras, métodos o procedimientos que el texto detalla sobre cómo 
se realiz

In [15]:
task1_zero_test['output'][1]

{'WHO': ['La doctora Margarite Griesz-Brisson'],
 'WHAT': ['una especialista alemana en neurofisiología y neurotoxicología']}

In [16]:
task1_zero_test['tags'][1]

[{'Tag_Start': 0,
  'Tag_End': 35,
  '5W1H_Label': 'WHO',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'La doctora Margarite Griesz-Brisson'},
 {'Tag_Start': 39,
  'Tag_End': 101,
  '5W1H_Label': 'WHAT',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'una especialista alemana en neurofisiología y neurotoxicología'}]

In [17]:
print(task1_zero_test['input'][2])

### Instrucciones:
Tu tarea es analizar el texto proporcionado para identificar y extraer fragmentos significativos, 
luego asignar a cada fragmento una etiqueta que describa la naturaleza de la información que contiene. 
Los resultados deben ser presentados en formato JSON.

Categorías de Etiquetado
Cada fragmento extraído debe clasificarse en una de las siguientes categorías:

WHAT (Qué): Señala los hechos, objetos, o cualquier elemento concreto mencionado en el texto.

WHO (Quién): Identifica a los sujetos o entidades (personas, organizaciones, etc.) que están 
involucrados o mencionados en el texto.

WHEN (Cuándo): Extrae detalles que especifican momentos o periodos de tiempo mencionados en el texto.

WHERE (Dónde): Localiza los lugares, espacios geográficos o direcciones mencionadas en el texto.

WHY (Por qué): Identifica las causas, razones o motivaciones explicadas en el texto.

HOW (Cómo): Describe las maneras, métodos o procedimientos que el texto detalla sobre cómo 
se realiz

In [18]:
task1_zero_test['output'][2]

{'WHAT': ['Los síntomas de alerta agudos',
  'dolores de cabeza, somnolencia, mareos, problemas de concentración y tiempo de reacción retardado, que son reacciones del sistema cognitivo']}

In [19]:
task1_zero_test['tags'][2]

[{'Tag_Start': 0,
  'Tag_End': 29,
  '5W1H_Label': 'WHAT',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'Los síntomas de alerta agudos'},
 {'Tag_Start': 34,
  'Tag_End': 173,
  '5W1H_Label': 'WHAT',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'dolores de cabeza, somnolencia, mareos, problemas de concentración y tiempo de reacción retardado, que son reacciones del sistema cognitivo'}]

# Create honest training dataset (train split)

In [15]:
# Load split train
task1_train = pd.read_json('Flares-dataset/5w1h_subtarea_1_train_train.json', lines=True)
task1_test = pd.read_json('Flares-dataset/5w1h_subtarea_1_train_test.json', lines=True)

In [16]:
task1_train[task1_train['Id'] == 1008]['Text']

601    Esta es una molécula de señalización inmunitaria asociada con una reacción inmune exagerada, la denominada «tormenta de citocinas».
Name: Text, dtype: object

In [17]:
task1_test['Text'][250]

'El experto ha acompañado el mensaje con una fotografía de uno de los casos observados, en el que se ve una lengua manchada o descolorida.\xa0'

In [18]:
task1_zero_train = format_data_zero(task1_train)
task1_zero_test = format_data_zero(task1_test)

In [19]:
task1_zero_train[0].keys()

dict_keys(['input', 'output', 'tags', 'Id'])

In [58]:
import os

# Create the directory if it does not exist
os.makedirs("data_files/json_format2_spanish", exist_ok=True)

In [59]:
import json

# Define the file path where you want to save the JSON data
file_path = 'data_files/json_format2_spanish/5w1h_subtask_1_zero_train_train_json_format2_spanish.json'

# Save the `formatted_data` as a JSON file
with open(file_path, 'w') as f:
    json.dump(task1_zero_train, f, ensure_ascii = False, indent=4) 

In [60]:
import json

# Define the file path where you want to save the JSON data
file_path = 'data_files/json_format2_spanish/5w1h_subtask_1_zero_train_test_json_format2_spanish.json'

# Save the `formatted_data` as a JSON file
with open(file_path, 'w') as f:
    json.dump(task1_zero_test, f, indent=4) 

In [61]:
task1_zero_train = pd.read_json('data_files/json_format2_spanish/5w1h_subtask_1_zero_train_train_json_format2_spanish.json')
task1_zero_test = pd.read_json('data_files/json_format2_spanish/5w1h_subtask_1_zero_train_test_json_format2_spanish.json')

In [62]:
task1_zero_train.shape

(1268, 4)

In [63]:
task1_zero_test[task1_zero_test['Id'] == 956]

,input,output,tags,Id
250,"A continuación, se presenta una instrucción que describe una tarea, junto con una entrada que proporciona contexto adicional. Escribe una respuesta que complete adecuadamente la solicitud.\n\n### Instrucciones:\nTu tarea es analizar el texto proporcionado y extraer fragmentos que pertenezcan a las categorías específicas, asegurándote de que los fragmentos sean secuencias continuas de palabras exactamente como aparecen en el texto, sin reordenamientos ni omisiones.\n\n1. Leer el Texto Completo:\nComprende el contenido general y el contexto del texto dado. Asegúrate de entender los principales eventos, descripciones, y entidades mencionadas.\n\n2. Extraer Fragmentos Relevantes:\nIdentifica y extrae partes del texto que corresponden a cada una de las siguientes categorías. Los fragmentos deben ser exactos y no alterados del texto original.\n\n3. Clasificar cada Fragmento:\nUtiliza las categorías predefinidas para anotar cada fragmento extraído. Asegúrate de que la clasificación sea precisa según la definición de cada categoría.\n\nCategorías:\n\nWHAT (Qué): Hechos, objetos, o cualquier elemento concreto mencionado.\nWHO (Quién): Sujetos o entidades (personas, organizaciones, etc.) mencionados.\nWHEN (Cuándo): Momentos o periodos de tiempo especificados.\nWHERE (Dónde): Lugares, espacios geográficos o direcciones.\nWHY (Por qué): Causas, razones o motivaciones.\nHOW (Cómo): Maneras, métodos o procedimientos detallados.\n\n### Información de Entrada:\nEl experto ha acompañado el mensaje con una fotografía de uno de los casos observados, en el que se ve una lengua manchada o descolorida.","{'WHO': ['El experto'], 'WHAT': ['el mensaje', 'una lengua manchada o descolorida'], 'HOW': ['con una fotografía de uno de los casos observados']}","[{'Tag_Start': 0, 'Tag_End': 10, '5W1H_Label': 'WHO', 'Reliability_Label': 'confiable', 'Tag_Text': 'El experto'}, {'Tag_Start': 25, 'Tag_End': 35, '5W1H_Label': 'WHAT', 'Reliability_Label': 'confiable', 'Tag_Text': 'el mensaje'}, {'Tag_Start': 36, 'Tag_End': 85, '5W1H_Label': 'HOW', 'Reliability_Label': 'semiconfiable', 'Tag_Text': 'con una fotografía de uno de los casos observados'}, {'Tag_Start': 103, 'Tag_End': 136, '5W1H_Label': 'WHAT', 'Reliability_Label': 'semiconfiable', 'Tag_Text': 'una lengua manchada o descolorida'}]",956


## Dataset content

In [64]:
print(task1_zero_train['input'][0])

A continuación, se presenta una instrucción que describe una tarea, junto con una entrada que proporciona contexto adicional. Escribe una respuesta que complete adecuadamente la solicitud.

### Instrucciones:
Tu tarea es analizar el texto proporcionado y extraer fragmentos que pertenezcan a las categorías específicas, asegurándote de que los fragmentos sean secuencias continuas de palabras exactamente como aparecen en el texto, sin reordenamientos ni omisiones.

1. Leer el Texto Completo:
Comprende el contenido general y el contexto del texto dado. Asegúrate de entender los principales eventos, descripciones, y entidades mencionadas.

2. Extraer Fragmentos Relevantes:
Identifica y extrae partes del texto que corresponden a cada una de las siguientes categorías. Los fragmentos deben ser exactos y no alterados del texto original.

3. Clasificar cada Fragmento:
Utiliza las categorías predefinidas para anotar cada fragmento extraído. Asegúrate de que la clasificación sea precisa según la d

In [65]:
task1_zero_train['output'][0]

{'WHO': ['el digital OK Diario', 'el Gobierno de España'],
 'WHERE': ['en un artículo que puedes leer en este enlace'],
 'HOW': ['con 45.980 euros'],
 'WHAT': ['un estudio sobre el impacto sexista de los piropos']}

In [66]:
task1_zero_train['tags'][0]

[{'Tag_Start': 23,
  'Tag_End': 43,
  '5W1H_Label': 'WHO',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'el digital OK Diario'},
 {'Tag_Start': 45,
  'Tag_End': 90,
  '5W1H_Label': 'WHERE',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'en un artículo que puedes leer en este enlace'},
 {'Tag_Start': 92,
  'Tag_End': 113,
  '5W1H_Label': 'WHO',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'el Gobierno de España'},
 {'Tag_Start': 131,
  'Tag_End': 147,
  '5W1H_Label': 'HOW',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'con 45.980 euros'},
 {'Tag_Start': 148,
  'Tag_End': 198,
  '5W1H_Label': 'WHAT',
  'Reliability_Label': 'semiconfiable',
  'Tag_Text': 'un estudio sobre el impacto sexista de los piropos'}]

In [67]:
task1_zero_test['Id']

0      1382
1       553
2       567
3       483
4       938
       ... 
312    1447
313     583
314     766
315    1097
316     789
Name: Id, Length: 317, dtype: int64

In [68]:
print(task1_zero_train['input'][1])

A continuación, se presenta una instrucción que describe una tarea, junto con una entrada que proporciona contexto adicional. Escribe una respuesta que complete adecuadamente la solicitud.

### Instrucciones:
Tu tarea es analizar el texto proporcionado y extraer fragmentos que pertenezcan a las categorías específicas, asegurándote de que los fragmentos sean secuencias continuas de palabras exactamente como aparecen en el texto, sin reordenamientos ni omisiones.

1. Leer el Texto Completo:
Comprende el contenido general y el contexto del texto dado. Asegúrate de entender los principales eventos, descripciones, y entidades mencionadas.

2. Extraer Fragmentos Relevantes:
Identifica y extrae partes del texto que corresponden a cada una de las siguientes categorías. Los fragmentos deben ser exactos y no alterados del texto original.

3. Clasificar cada Fragmento:
Utiliza las categorías predefinidas para anotar cada fragmento extraído. Asegúrate de que la clasificación sea precisa según la d

In [69]:
task1_zero_train['output'][1]

{'WHAT': ['una sustancia soluble'],
 'WHERE': ['en agua', 'en ambientes de baja humedad'],
 'HOW': ['al cocinar alimentos ricos en hidratos de carbono -como cereales o patatas- a temperaturas superiores a 120 grados'],
 'WHEN': ['cuando freímos, tostamos u horneamos']}

In [70]:
task1_zero_train['tags'][1]

[{'Tag_Start': 12,
  'Tag_End': 33,
  '5W1H_Label': 'WHAT',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'una sustancia soluble'},
 {'Tag_Start': 34,
  'Tag_End': 41,
  '5W1H_Label': 'WHERE',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'en agua'},
 {'Tag_Start': 55,
  'Tag_End': 169,
  '5W1H_Label': 'HOW',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'al cocinar alimentos ricos en hidratos de carbono -como cereales o patatas- a temperaturas superiores a 120 grados'},
 {'Tag_Start': 172,
  'Tag_End': 200,
  '5W1H_Label': 'WHERE',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'en ambientes de baja humedad'},
 {'Tag_Start': 213,
  'Tag_End': 249,
  '5W1H_Label': 'WHEN',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'cuando freímos, tostamos u horneamos'}]

In [71]:
print(task1_zero_train['input'][2])

A continuación, se presenta una instrucción que describe una tarea, junto con una entrada que proporciona contexto adicional. Escribe una respuesta que complete adecuadamente la solicitud.

### Instrucciones:
Tu tarea es analizar el texto proporcionado y extraer fragmentos que pertenezcan a las categorías específicas, asegurándote de que los fragmentos sean secuencias continuas de palabras exactamente como aparecen en el texto, sin reordenamientos ni omisiones.

1. Leer el Texto Completo:
Comprende el contenido general y el contexto del texto dado. Asegúrate de entender los principales eventos, descripciones, y entidades mencionadas.

2. Extraer Fragmentos Relevantes:
Identifica y extrae partes del texto que corresponden a cada una de las siguientes categorías. Los fragmentos deben ser exactos y no alterados del texto original.

3. Clasificar cada Fragmento:
Utiliza las categorías predefinidas para anotar cada fragmento extraído. Asegúrate de que la clasificación sea precisa según la d

In [72]:
task1_zero_train['output'][2]

{'WHO': ['el Ministro de Salud']}

In [73]:
task1_zero_train['tags'][2]

[{'Tag_Start': 225,
  'Tag_End': 245,
  '5W1H_Label': 'WHO',
  'Reliability_Label': 'confiable',
  'Tag_Text': 'el Ministro de Salud'}]

# Create Test dataset

#### Create a Prompt with JSON Output (json_format1):

{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], 'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}

In [ ]:
### Con ejemplo y en formato JSON
def format_data_zero(df):
    formatted_data = []  # List to hold the formatted dictionaries

    instruction = ("Tu tarea consiste en analizar el texto proporcionado y extraer fragmentos significativos, "
                   "transcribiéndolos exactamente como aparecen. Asigna a cada fragmento la etiqueta "
                   "correspondiente basada en la información que representa. Presenta los resultados en formato JSON. Las categorías de etiquetado son:"
                   "\n\nWHO: Sujetos o entidades involucradas.\nWHAT: Hechos u objetos mencionados.\nWHEN: Detalles relacionados con el tiempo.\n"
                   "WHERE: Lugares mencionados.\nWHY: Causas o razones.\nHOW: Maneras o métodos descritos."
                   "\n\nAbajo es un ejemplo:\n\nInput: Dos días, exactamente han pasado dos días desde que Sánchez compareciera en rueda de prensa en la Moncloa afirmando que a España llegarían, "
                   "entre abril y septiembre, un total de 87 millones de vacunas para darnos cuenta de que las mentiras de Sánchez hacen bueno ese refrán que dice que "
                   "“la mentira tiene las patas muy cortas”. Output: "
                   "{'WHO': ['Sánchez', 'a España', 'de Sánchez'], 'WHERE': ['en rueda de prensa en la Moncloa'], 'WHEN': ['entre abril y septiembre'], "
                   "'WHAT': ['un total de 87 millones de vacunas', 'las mentiras', 'ese refrán que dice']}\n\nAhora, completa la siguiente tarea:\n\nInput: ")

    for i in range(len(df['Text'])):
        # Constructing the 'input' part with instruction
        input_text = instruction + df['Text'][i]
        
        # Append the formatted dictionary to the list
        formatted_data.append({'input': input_text, 'Id':int(df['Id'][i])})

    return formatted_data

In [32]:
task1_test = pd.read_json('Flares-dataset/5w1h_subtarea_1_test.json', lines=True)

In [33]:
import os

# Create the directory if it does not exist
os.makedirs("data_files/json_format1", exist_ok=True)

In [34]:
task1_zero_test = format_data_zero(task1_test)

In [35]:
import json

# Define the file path where you want to save the JSON data
file_path = 'data_files/json_format1/5w1h_subtask_1_test_json_format1.json'

# Save the `formatted_data` as a JSON file
with open(file_path, 'w') as f:
    json.dump(task1_zero_test, f, ensure_ascii = False, indent=4) 